## Send parameter files to new case

This script sends a subset of parameter files from an existing case, or cases, to a new case or cases.

It sends just the parameter sets that are flags as "promising".

In [18]:
import pandas as pd
import glob
import os
import netCDF4 as nc4
import sys
sys.path.append('/glade/u/home/adamhb/Earth-System-Model-Tools')
import esm_tools
import re
import math
import shutil
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
np.set_printoptions(threshold=1000)
import seaborn as sns

In [15]:
path_to_aggreagated_data = '/glade/work/adamhb/processed_output/CZ2_equilibrium_700yrs_042524_01_-17e2acb6a_FATES-1449c787/equilibrium_700yrs_042524_metrics_and_params.csv'

## Functions

In [16]:
def copy_and_rename_param_files(param_file_paths,inst_per_new_case, new_param_subdir_base_name,
                                param_subdir_root = '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles'):
    '''
    This is a function that takes a list of successful ensemble members, read's the file path to the parameter file of each successful ensemble member,
    and moves these old param files into new param file subdirs then creates a new param file subdir
    '''
    param_file_paths = np.array(param_file_paths)
    inst_per_new_case = min(inst_per_new_case,len(param_file_paths))

    path_to_ensemble_prov = '/glade/u/home/adamhb/ensemble_provenance_logs'
    n_cases = math.ceil(len(param_file_paths) / inst_per_new_case)
    case_numbers = [str(i).zfill(2) for i in range(1,n_cases + 1)]
    new_param_subdir_base_path = os.path.join(param_subdir_root,new_param_subdir_base_name)
    new_param_subdir_paths = [new_param_subdir_base_path + "_" + case_number for case_number in case_numbers]
    [os.mkdir(p) for p in new_param_subdir_paths if os.path.exists(p) == False]

    dst_file_paths = []
    for case_number in case_numbers:
        for inst_tag in [str(i).zfill(4) for i in range(1,inst_per_new_case + 1)]:
            dst_file_paths.append(f'{param_subdir_root}/{new_param_subdir_base_name}_{case_number}/ca_5pfts_100523_{inst_tag}.nc')

    if len(dst_file_paths) != len(param_file_paths):
        print("srcn",len(param_file_paths))
        print("dstn",len(dst_file_paths))
        print("Error in number of dst files")
        return

    pd.DataFrame({'src':param_file_paths,'dst':dst_file_paths}).to_csv(f'{path_to_ensemble_prov}/{new_param_subdir_base_name}_provenence.csv')

    for i in range(len(param_file_paths)):
        print("Copying",param_file_paths[i],"to",dst_file_paths[i])
        shutil.copy(param_file_paths[i],dst_file_paths[i])

In [19]:
df = pd.read_csv(path_to_aggreagated_data)
params_for_next_sim = esm_tools.inst_to_tag(np.array(df.loc[df['promising'] == True]['param_file_path']))
print(params_for_next_sim)
copy_and_rename_param_files(params_for_next_sim,128, "test_700yrs_043024",
                                param_subdir_root = '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles')

['/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/equilibrium_700yrs_042524_01/ca_5pfts_100523_0011.nc', '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/equilibrium_700yrs_042524_01/ca_5pfts_100523_0024.nc', '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/equilibrium_700yrs_042524_01/ca_5pfts_100523_0048.nc', '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/equilibrium_700yrs_042524_01/ca_5pfts_100523_0054.nc', '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/equilibrium_700yrs_042524_01/ca_5pfts_100523_0064.nc', '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/equilibrium_700yrs_042524_01/ca_5pfts_100523_0065.nc', '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/equilibrium_700yrs_042524_01/ca_5pfts_100523_0067.nc', '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/equilibrium_700yrs_042524_01/ca_5pfts_100523_0092.nc']
Copying /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/equilibrium_700yrs_042524_01/ca_5pfts_100523_001